In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

Reading data

In [2]:
loan = pd.read_csv('loan.csv')

C:\Users\anant\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Creating dataframe with required columns

In [3]:
loan = loan[loan['title'].notnull()]

In [4]:
l_c=['title','loan_status']
loan=loan[l_c]
loan.shape

(126067, 2)

Assigning loan Statuses to create a binary division of good and bad loans

In [5]:
loan['loan_status'] = loan['loan_status'].map({'Charged Off': 1, 'Fully Paid': 1,'Current':np.NaN,'Does not meet the credit policy. Status:Fully Paid':0,'Does not meet the credit policy. Status:Charged Off':0,'In Grace Period':0,'Late (16-30 days)':0,'Late (31-120 days)':0})
loan.shape

(126067, 2)

Prepared data for NLP

In [6]:
loan.dropna(axis=0,inplace=True)
loan.shape

(125763, 2)

In [7]:
loan['loan_status'].unique()

array([1., 0.])

Tokenizing

In [8]:
loan['title_tok']=loan.apply(lambda row: nltk.word_tokenize(row['title']), axis=1)

Removing stop words and punctuations

In [9]:
useless_words = nltk.corpus.stopwords.words('english')+list(string.punctuation)
loan['tok']=loan.apply(lambda row:[word for word in row['title_tok'] if not word in useless_words], axis=1)

Stemming

In [10]:
ps = PorterStemmer()
loan['stem']=loan.apply(lambda row:[ps.stem(word) for word in row['tok']], axis=1)

In [11]:
ps = PorterStemmer()
loan['last_out']=loan.apply(lambda row:[''.join(word) for word in row['stem'] if word not in string.punctuation],axis=1)

Creating final dataframe for NLP

In [12]:
loan['out'] = loan.last_out.map(lambda x: ' '.join(x))

In [13]:
loan=loan[['loan_status','out']]

Utilizing CountVectorizer

In [14]:
count_vect = CountVectorizer()  
counts = count_vect.fit_transform(loan['out'])

Creating Naive Bayes Model to predict status of loan from title

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(counts, loan['loan_status'], test_size=0.25, random_state=0)

from sklearn import metrics

model = MultinomialNB().fit(X_train, y_train)
predicted=model.predict(X_test)
predicted_train=model.predict(X_train)

print("Accuracy_rate_train:",metrics.accuracy_score(y_train, predicted_train))

print("Accuracy_rate_test:",metrics.accuracy_score(y_test, predicted))

Accuracy_rate_train: 0.9773329657980111
Accuracy_rate_test: 0.9729016252663719
